# Data imputation

## Time series

In [ ]:
import impyute
from tqdm.notebook import tqdm

In [ ]:
from imblearn.over_sampling import ADASYN, SMOTE
from sklearn.feature_selection import SelectKBest, f_regression, chi2, f_classif
from sklearn.metrics import f1_score, mean_squared_error, accuracy_score, r2_score, roc_auc_score, recall_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression, BayesianRidge, LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import Lasso
from sklearn.svm import SVC, LinearSVC

import xgboost as xgb

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
IDENTIFIERS = ["pid", "Time", "Age"]
MEDICAL_TESTS = [
    "LABEL_BaseExcess",
    "LABEL_Fibrinogen",
    "LABEL_AST",
    "LABEL_Alkalinephos",
    "LABEL_Bilirubin_total",
    "LABEL_Lactate",
    "LABEL_TroponinI",
    "LABEL_SaO2",
    "LABEL_Bilirubin_direct",
    "LABEL_EtCO2",
]
VITAL_SIGNS = ["LABEL_RRate", "LABEL_ABPm", "LABEL_SpO2", "LABEL_Heartrate"]
SEPSIS = ["LABEL_Sepsis"]
ESTIMATOR = {"bayesian": BayesianRidge(), "decisiontree": DecisionTreeRegressor(max_features="sqrt", random_state=0), 
                "extratree": ExtraTreesRegressor(n_estimators=10, random_state=0), 
                "knn": KNeighborsRegressor(n_neighbors=10, weights="distance")}
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
def sigmoid_f(x):
    """To get predictions as confidence level, the model predicts for all 12 sets of measures for
    each patient a distance to the hyperplane ; it is then transformed into a confidence level using
    the sigmoid function ; the confidence level reported is the mean of all confidence levels for a
    single patient

    Args:
        x (float): input of the sigmoid function

    Returns:
       float: result of the sigmoid computation.

    """
    return 1 / (1 + np.exp(-x))

In [ ]:
# download data 
df_train = pd.read_csv(r"data/train_features.csv")
df_train_label = pd.read_csv(r"data/train_labels.csv")
df_val = pd.read_csv(r"data/test_features.csv")

In [ ]:
df_train_mean = df_train.copy().fillna(df_train.mean())
df_test_mean = df_val.copy().fillna(df_val.mean())

In [ ]:
PRESENCE_IND = ['EtCO2','Lactate','Bilirubin_direct', 'Bilirubin_total', 'TroponinI',  'PaCO2', 'FiO2', 'AST', 'SaO2']
MEAN_IND= ['BUN', 'PTT', 'Hgb', 'HCO3', 'Alkalinephos', 'Chloride', 'Hct', 'Fibrinogen', 'Phosphate', 'WBC', 'Creatinine', 
              'Platelets', 'Glucose', 'Magnesium', 'Potassium', 'Calcium', 'pH', 'BaseExcess']
TIME_IND = ['Heartrate', 'Temp', 'SpO2', 'ABPs', 'RRate', 'ABPm', 'ABPd']

From visual analysis, it seems like we should treat the columns in the following way:
* EtCO2, Lactate, Bilirubin_direct, Bilirubin_total, TroponinI, PaCO2, FiO2, AST, SaO2 : presence (etco2 for acute respiratory distress, lactate is indicator of sepsis shock, bilirubin for liver diseases, TroponinI if you've had heart issues, etc)
* BUN, PTT, Hgb, HCO3, Alkalinephos, chloride, hct, fibrinogen, phosphate, WBC, creatinine, platelet, glucose, magnesium, potassium, calcium: set all missing values to mean of existing values (these are all blood test indicators)
* pH, base excess : set all missing values to mean of existing values (these values should be fully correlated)
* Heartrate, Temp, SpO2, ABPs, RRate, ABPm, ABPd : time series imputation, LCOF 


In [ ]:
df_train_copy = df_train.copy().head(36000)

In [ ]:
def transform_df(df, PRESENCE_IND, MEAN_IND, TIME_IND):
    """Performs data imputation and feature transformation in the following manner : for all data in PRESENCE_IND, 
    adds a column with the presence or absence for each patient of a test feature; for all data in PRESENCE_IND and MEAN_IND,
    if there are any values for a patient, replaces the NaN by the mean of that patient; for data in TIME_IND, does linear 
    interpolation of values to fill the time series
    
    Args: df (pandas.core.DataFrame): dataframe to transform
        PRESENCE_IND (list): features for presence transformation
        MEAN_IND (list): features for mean transformation
        TIME_IND (list): features for time series interpolation
    
    Returns: df (pandas.core.DataFrame): transformed dataframe
    """
    for pid in tqdm(df["pid"].unique()):
        pdf = df.loc[df["pid"]==pid]
        for col in PRESENCE_IND:
            if not(pd.isnull(pdf[col]).all()):
                df.loc[df["pid"]==pid, col+"_presence"]=1
            else:
                df.loc[df["pid"]==pid, col+"_presence"]=0
        for col in MEAN_IND+PRESENCE_IND:
            if not(pd.isnull(pdf[col]).all()):
                df.loc[df["pid"]==pid, col]=df.loc[df["pid"]==pid, col].fillna(df.loc[df["pid"]==pid, col].mean())
        for col in TIME_IND:
            if not(pd.isnull(pdf[col]).all()):
                df.loc[df["pid"]==pid, col]=df.loc[df["pid"]==pid, col].interpolate()
                df.loc[df["pid"]==pid, col]=df.loc[df["pid"]==pid, col].fillna(method='ffill')
                df.loc[df["pid"]==pid, col]=df.loc[df["pid"]==pid, col].fillna(method='bfill')
    return df     

In [ ]:
# transform the training data
df_train_copy = transform_df(df_train_copy, PRESENCE_IND, MEAN_IND, TIME_IND)

In [ ]:
# construct the training labels
df_train_label_copy = df_train_label.set_index(["pid"])
df_label = df_train_label_copy.loc[df_train_copy["pid"].unique().astype(int)]
df_label = df_label.reset_index()
df_label

In [ ]:
df_test_copy = df_val.copy().head(6000)

In [ ]:
df_test_copy = transform_df(df_test_copy, PRESENCE_IND, MEAN_IND, TIME_IND)

Then, we take the median of all these values for the 12h of the patient to get one line per patient for tasks 1 & 2, and keep all values for task 3

In [ ]:
df_train_1_2 = df_train_copy.groupby(["pid"], as_index=False).median()
df_train_3 = df_train_copy[IDENTIFIERS + TIME_IND]
df_test_1_2 = df_test_copy.groupby(["pid"], as_index=False).median()
df_test_3 = df_test_copy[IDENTIFIERS + TIME_IND]

Finally, we use a data imputer to solve values that are still NaN

In [ ]:
def perform_imputation(df_train,df_test):
    columns = df_train.columns
    imputer = IterativeImputer(
        estimator=KNeighborsRegressor(),
        missing_values=np.nan,
        sample_posterior=False,
        max_iter=10,
        tol=0.001,
        n_nearest_features=None, # Meaning all features are used
        initial_strategy='median',
        imputation_order='descending',
        skip_complete=False,
        min_value=None,
        max_value=None,
        verbose=2,
        random_state=42,
        add_indicator=False
    )

    df_train = imputer.fit_transform(df_train.values)
    df_train = pd.DataFrame(df_train, columns=columns)
    df_train['pid'] = df_train['pid'].astype(int)
    df_test = imputer.transform(df_test.values)
    df_test = pd.DataFrame(df_test, columns=columns)
    df_test['pid'] = df_test['pid'].astype(int)
    return df_train,df_test

In [ ]:
df_train_1_2, df_test_1_2 = perform_imputation(df_train_1_2, df_test_1_2)

In [ ]:
df_train_3, df_test_3 = perform_imputation(df_train_3, df_test_3)

In [ ]:
def data_formatting(df_train, df_test, df_label):
    # Data formatting
    X_train= df_train.drop(columns=["pid","Time"]).values
    X_test = df_test.drop(columns=["pid","Time"]).values
    # Create list with different label for each medical test
    print("Creating a list of labels for each medical test")
    y_train_medical_tests = []
    for test in MEDICAL_TESTS:
        y_train_medical_tests.append(df_label[test].astype(int).values)

    # Create list with different label for sepsis
    print("Creating a list of labels for sepsis")
    y_train_sepsis = []
    for sepsis in SEPSIS:
        y_train_sepsis.append(df_label[sepsis].astype(int).values)
    return X_train,X_test,y_train_medical_tests,y_train_sepsis

In [ ]:
X_train_1_2, X_test_1_2, y_train_medical_tests,y_train_sepsis = \
data_formatting(df_train_1_2,df_test_1_2,df_label)

In [ ]:
X_train_mean, X_test_mean, y_train_medical_tests_mean, y_train_sepsis_mean = \
    data_formatting(df_train_mean,df_test_mean,df_train_label)

In [ ]:
for i in range(len(y_train_medical_tests_mean)):
    newy = []
    for j in range(len(y_train_medical_tests_mean[i])):
        newy+=[y_train_medical_tests_mean[i][j] for k in range(12)]
    y_train_medical_tests_mean[i]=np.array(newy)

# Task 1

In [ ]:
# Scale data 
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled = scaler.fit_transform(X_train_1_2)
X_test_scaled = scaler.transform(X_test_1_2)

In [ ]:
# Scale data 
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_scaled_mean = scaler.fit_transform(X_train_mean)
X_test_scaled_mean = scaler.transform(X_test_mean)

In [ ]:
# Modelling of medical tests using SVC
models = []
scores = []
columns_medical_tests = []
for i, test in enumerate(MEDICAL_TESTS):
    print("Resampling data")
    sampler = ADASYN()
    X_train, y_train = sampler.fit_sample(X_train_scaled_mean, y_train_medical_tests_mean[i])
    print(f"Fitting model for {test}.")
    X_train, X_test, y_train, y_test = train_test_split(
        X_train, y_train, test_size=0.20, random_state=42, shuffle=True
    )

    feature_selector = SelectKBest(score_func=f_classif, k=3)
    X_train = feature_selector.fit_transform(X_train, y_train)
    X_test = feature_selector.transform(X_test)
    columns = feature_selector.get_support(indices=True)
    columns_medical_tests.append(columns)

    clf = SVC(C=0.01, kernel="poly", degree=4, class_weight="balanced").fit(X_train, y_train)
    models.append(clf)
    y_pred = clf.decision_function(X_test)
    y_pred = [sigmoid_f(y_pred[i]) for i in range(len(y_pred))]
    scores.append(roc_auc_score(y_test, y_pred))
print(f"Scores {scores}")
print(f"Average score {np.mean(scores)}")

In [ ]:
# Modelling of medical tests using logistic regression with cross validation
models = []
scores = []
columns_medical_tests = []
for i, test in enumerate(MEDICAL_TESTS):
    print("Resampling data")
    sampler = ADASYN()
    X_train, y_train = sampler.fit_sample(X_train_scaled, y_train_medical_tests[i])
    print(f"Fitting model for {test}.")
    X_train, X_test, y_train, y_test = train_test_split(
        X_train, y_train, test_size=0.20, random_state=42, shuffle=True
    )

    feature_selector = SelectKBest(score_func=f_classif, k=3)
    X_train = feature_selector.fit_transform(X_train, y_train)
    X_test = feature_selector.transform(X_test)
    columns = feature_selector.get_support(indices=True)
    columns_medical_tests.append(columns)
    clf = LogisticRegressionCV(cv=5, random_state=42).fit(X_train, y_train)
    models.append(clf)
    scores.append(roc_auc_score(y_test, clf.predict_proba(X_test)[:,1]))
print(f"Scores {scores}")
print(f"Average score {np.mean(scores)}")

In [ ]:
# using XGBoost
clf = xgb.XGBClassifier(objective="binary:logistic", n_thread=-1)

scores = []
for i, test in enumerate(MEDICAL_TESTS):
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_train_scaled, y_train_medical_tests[i], test_size=0.20, random_state=42, shuffle=True
    )
    
    print("Resampling data")
    sampler = ADASYN()
    X_train, y_train = sampler.fit_sample(X_train, y_train)
    print(f"Fitting model for {test}.")


    label_counts = pd.Series(y_train).value_counts()
    scale_pos_weight = label_counts[0] / label_counts[1]
    if scale_pos_weight < 2: scale_pos_weight = 2
    
    # Coarse parameter grid not optimized at all yet
    param_grid = {
        "booster": ["dart"],
        "eta": [0.1,0.01, 0.001],
        "min_child_weight": range(1, 10, 1),
        "max_depth": range(3, 8, 1),
        "gamma": range(0, 100, 2),
        "max_delta_step": range(3, 10, 1),
        "subsample": np.arange(0.1, 1, 0.05),
        "colsample_bytree": np.arange(0.3, 1, 0.05),
        "n_estimators": range(50, 120, 2),
        "scale_pos_weight": [int(scale_pos_weight)],
        "reg_lambda": [100], # Ridge regularization
        "reg_alpha": [0], # Lasso regularization
        "eval_metric": ["error"]
    }
    
    feature_selector = SelectKBest(score_func=f_classif, k=3)
    X_train = feature_selector.fit_transform(X_train, y_train)
    X_test = feature_selector.transform(X_test)
    columns = feature_selector.get_support(indices=True)
    columns_medical_tests.append(columns)

    coarse_search = RandomizedSearchCV(estimator=clf,
            param_distributions=param_grid, scoring="roc_auc",
            n_jobs=-1, cv=5, n_iter=10, verbose=1)
    coarse_search.fit(X_train, y_train)
    models.append(coarse_search.best_estimator_)
    scores.append(roc_auc_score(y_test, coarse_search.best_estimator_.predict_proba(X_test)[:,1]))
print(f"Scores {scores}")
print(f"Average score {np.mean(scores)}")

# Task 2

In [ ]:
# Model and predict sepsis

X_train, X_test, y_train, y_test = train_test_split(
    X_train_scaled, y_train_sepsis[0], test_size=0.20, random_state=42, shuffle=True
)
print("Resampling data")
sampler = ADASYN()
X_train, y_train = sampler.fit_sample(X_train, y_train)


print("Applying feature selection")
feature_selector = SelectKBest(score_func=f_classif, k=3)
X_train = feature_selector.fit_transform(X_train, y_train)
X_test = feature_selector.transform(X_test)
columns_sepsis = feature_selector.get_support(indices=True)

models, scores = [], []
for i, C in enumerate(np.linspace(0.0001, 0.01, num=5)):
    print("Starting SVC for C={}".format(C))
    models.append(
        SVC(C=C, kernel="poly", class_weight="balanced").fit(X_train, y_train)
    )
    y_pred = models[i].decision_function(X_test)
    y_pred = [sigmoid_f(y_pred[i]) for i in range(len(y_pred))]
    scores.append(roc_auc_score(y_test, y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, np.around(y_pred)).ravel()
    size = len(y_test)
    print(
        "The ROC AUC score is {}, the TPrate is {}%, the FPrate is {}% "
        "for iteration {}".format(scores[i], tp/(tp+fn)*100, tn/(tn+fp)*100, i)
    )
    print(y_test[:30],np.around(y_pred[:30]))
best = np.argmax(scores)
model, score = models[best], scores[best]

# Task 3

In [ ]:
TREND_LABEL = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

In [ ]:
pd.set_option('precision', 3)

In [ ]:
def train_lasso(df_train_3, df_train_label, test, label_test, alpha):
    X_train, y_train = [],[]
    for pid in df_train_3["pid"].unique():
        X_train.append(df_train_3.loc[df_train_3["pid"]==pid, test].values)
        y_train.append(df_train_label.loc[df_train_label["pid"]==pid, label_test].values)
    X_train, y_train = np.array(X_train),np.array(y_train)
    X_train, X_test, y_train, y_test = train_test_split(
        X_train, y_train, test_size=0.20, random_state=42, shuffle=True
    )
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_train, y_train)
    print("R2 score for lasso model using innate scoring: ",lasso_model.score(X_train, y_train))
    y_pred = lasso_model.predict(X_test)
    print("R2 score on test set : ",r2_score(y_test, y_pred))
    print(f"Diff is {np.around(np.abs([(y_pred[i]-y_test[i])/y_test[i] for i in range(len(y_pred))][:10]),2)}")
    return lasso_model

In [ ]:
def predict_lasso(lasso_model, df_test_3, test):
    X_test = []
    for pid in df_test_3["pid"].unique():
        X_test.append(df_test_3.loc[df_test_3["pid"]==pid, test].values)
    predictions = lasso_model.predict(X_test)
    return predictions

In [ ]:
def lasso_subtask_3(df_train_3, df_train_label, TREND_LABEL):
    lasso_models = []
    for label_test in TREND_LABEL:
        print('Testing model for {}'.format(label_test))
        test = label_test.split("_")[1]
        lasso_models.append(train_lasso(df_train_3, df_train_label, test, label_test, 0.0001))
    return lasso_models

In [ ]:
lasso_models = lasso_subtask_3(df_train_3, df_label, TREND_LABEL)

In [ ]:
def get_predictions_3(df_test_3, lasso_models, TREND_LABEL):
    predictions = []
    for i in range(len(lasso_models)):
        test = TREND_LABEL[i].split("_")[1]
        predictions.append(predict_lasso(lasso_models[i], df_test_3, test))
    predictions = np.array(predictions).T
    predictions = pd.DataFrame(data=predictions, columns=TREND_LABEL, index=(df_test_3["pid"].unique()).astype(int))
    return(predictions)

In [ ]:
predictions = get_predictions_3(df_test_3, lasso_models, TREND_LABEL)

In [ ]:
predictions

Discoveries: we already have the values of some we want to predict in the test set, so time series modelling? Prophet API possibly?